In [1]:
# !pip install -U psycopg2-binary
# !pip install -U jsondiff

In [2]:
import psycopg2
import jsondiff
import sqlite3
import json
import os
from datetime import date, datetime
import csv

with open('./config.json') as f:
    config = json.load(f)

db_file = './sierra_patron_data.db'

class SQLiteConnection:
    """
    Usage:
    with SQLiteConnection('your_database_path.sqlite') as conn:
        # Your database operations here
    """
    def __init__(self, db_path):
        self.db_path = db_path

    @staticmethod
    def compute_diff(new_json, old_json):
        return str(
            jsondiff.diff(
                new_json, 
                old_json, 
                load=True, 
                dump=True, 
                marshal=True, 
                syntax='explicit'
            )
        )

    def __enter__(self):
        self.conn = sqlite3.connect(self.db_path)
        self.conn.create_function('json_diff', 2, self.compute_diff)
        return self.conn

    def __exit__(self, exc_type, exc_val, exc_tb):
        sql = """\
        PRAGMA analysis_limit=2000;
        PRAGMA optimize;
        """
        self.cursor = self.conn.cursor()
        self.cursor.executescript(sql)
        
        self.conn.close()

# some test suggestions
# 
# import unittest
# import sqlite3

# class TestSQLiteConnection(unittest.TestCase):

#     TEST_DB_PATH = ':memory:'  # We'll use an in-memory database for testing

#     def test_connection_established(self):
#         with SQLiteConnection(self.TEST_DB_PATH) as conn:
#             self.assertIsInstance(conn, sqlite3.Connection)

#     def test_json_diff_function_registered(self):
#         with SQLiteConnection(self.TEST_DB_PATH) as conn:
#             cursor = conn.cursor()
#             # We'll use a simple test for the json_diff function here
#             result = cursor.execute("SELECT json_diff('old', 'new')").fetchone()
#             self.assertEqual(result[0], "old -> new")

#     def test_connection_closed_after_exit(self):
#         with SQLiteConnection(self.TEST_DB_PATH) as conn:
#             pass  # Exiting the with block
#         self.assertTrue(conn.in_transaction is None)

# if __name__ == '__main__':
#     unittest.main()

# if not os.path.isfile(db_file):
#     print('not exists')
#     # start searching from timestamp 0
#     latest_update = datetime.fromtimestamp(0).date().isoformat()
# else:
#     # query the sqlite db for the last update date
#     pass



In [3]:
sql = """\
PRAGMA journal_mode=WAL;
PRAGMA cache_size = 10000;

CREATE TABLE IF NOT EXISTS patrons (
    patron_record_id INTEGER PRIMARY KEY,
    patron_record_num INTEGER,
    campus_code TEXT,
    barcode1 TEXT,
    home_library_code TEXT,
    ptype_code INTEGER,
    create_date DATE,
    delete_date DATE,
    update_date DATE,
    expire_date DATE,
    active_date DATE,
    claims_returned_total INTEGER,
    owed_amt_cents INTEGER,
    mblock_code TEXT,
    highest_level_overdue_num INTEGER,
    num_revisions INTEGER
);

CREATE TABLE IF NOT EXISTS patron_address_json (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    patron_record_id INTEGER UNIQUE,
    json_data TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (patron_record_id) REFERENCES patrons(patron_record_id)
);

CREATE TABLE IF NOT EXISTS identifiers_json (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    patron_record_id INTEGER UNIQUE,
    json_data TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (patron_record_id) REFERENCES patrons(patron_record_id)
);

CREATE TABLE IF NOT EXISTS phone_numbers_json (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    patron_record_id INTEGER UNIQUE,
    json_data TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (patron_record_id) REFERENCES patrons(patron_record_id)
);

CREATE TABLE IF NOT EXISTS emails_json (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    patron_record_id INTEGER UNIQUE,
    json_data TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (patron_record_id) REFERENCES patrons(patron_record_id)
);

CREATE TABLE IF NOT EXISTS json_changes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    json_table_name TEXT,
    patron_record_id INTEGER,
    diff TEXT,
    change_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);


CREATE TRIGGER IF NOT EXISTS compute_address_diff
AFTER UPDATE ON patron_address_json
FOR EACH ROW
WHEN OLD.json_data != NEW.json_data
BEGIN
    INSERT INTO json_changes (json_table_name, patron_record_id, diff)
    VALUES ('patron_address_json', NEW.patron_record_id, json_diff(NEW.json_data, OLD.json_data));
END;


CREATE TRIGGER IF NOT EXISTS compute_identifiers_diff
AFTER UPDATE ON identifiers_json
FOR EACH ROW
WHEN OLD.json_data != NEW.json_data
BEGIN
    INSERT INTO json_changes (json_table_name, json_table_id, diff)
    VALUES ('identifiers_json', NEW.patron_record_id, json_diff(NEW.json_data, OLD.json_data));
END;


CREATE TRIGGER IF NOT EXISTS compute_phone_numbers_diff
AFTER UPDATE ON phone_numbers_json
FOR EACH ROW
WHEN OLD.json_data != NEW.json_data
BEGIN
    INSERT INTO json_changes (json_table_name, json_table_id, diff)
    VALUES ('phone_numbers_json', NEW.patron_record_id, json_diff(NEW.json_data, OLD.json_data ));
END;


CREATE TRIGGER IF NOT EXISTS compute_emails_diff
AFTER UPDATE ON emails_json
FOR EACH ROW
WHEN OLD.json_data != NEW.json_data
BEGIN
    INSERT INTO json_changes (json_table_name, json_table_id, diff)
    VALUES ('emails_json', NEW.patron_record_id, json_diff(NEW.json_data, OLD.json_data));
END;

ANALYZE;
"""

with SQLiteConnection(db_file) as con:
    cursor = con.cursor()
    cursor.executescript(sql)

    cursor.close()

In [4]:
sql_patron_insert = """\
INSERT INTO patrons (
    patron_record_id, patron_record_num, campus_code, barcode1, home_library_code,
    ptype_code, create_date, delete_date, update_date, expire_date, active_date,
    claims_returned_total, owed_amt_cents, mblock_code, highest_level_overdue_num, num_revisions
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
ON CONFLICT(patron_record_id) DO UPDATE SET
    patron_record_num = excluded.patron_record_num,
    campus_code = excluded.campus_code,
    barcode1 = excluded.barcode1,
    home_library_code = excluded.home_library_code,
    ptype_code = excluded.ptype_code,
    create_date = excluded.create_date,
    delete_date = excluded.delete_date,
    update_date = excluded.update_date,
    expire_date = excluded.expire_date,
    active_date = excluded.active_date,
    claims_returned_total = excluded.claims_returned_total,
    owed_amt_cents = excluded.owed_amt_cents,
    mblock_code = excluded.mblock_code,
    highest_level_overdue_num = excluded.highest_level_overdue_num,
    num_revisions = excluded.num_revisions;
"""

sql_address_json_insert = """\
INSERT INTO patron_address_json (
    patron_record_id, json_data
) VALUES (?, ?)
ON CONFLICT(patron_record_id) DO UPDATE SET
    json_data = excluded.json_data
;
"""

sql_identifiers_json_insert = """\
INSERT INTO identifiers_json (patron_record_id, json_data) VALUES (?, ?);
"""

sql_phone_numbers_json_insert = """\
INSERT INTO phone_numbers_json (patron_record_id, json_data) VALUES (?, ?);
"""

sql_emails_json_insert = """\
INSERT INTO emails_json (patron_record_id, json_data) VALUES (?, ?);
"""

In [5]:
# connect to both the sierra db, and the local sqlite db
with psycopg2.connect(dsn=config['dsn']) as con, \
SQLiteConnection(db_file) as con_sqlite:
        
    cursor = con.cursor(name="named_cursor")
    cursor_sqlite = con_sqlite.cursor()

    # TODO .... query the local db or preserve a target a date with more precision than just the "date"
    # 
    # get the latest update from the sqlite table .. or default to the earlest date possible
    latest_update = datetime.fromtimestamp(0).date().isoformat()

    sql = """\
    SELECT
    	datetime(
    		max(json_changes.change_timestamp),   -- last record change detected
    		'-8 hours',	    -- just to be sure pad by a bunch
    		'localtime'	    -- dates in the sierra db are stored with gmt offset
    	) as lastupdate
    FROM
    	json_changes
    LIMIT 1
    """
    try:
        cursor_sqlite.execute(sql)
        result = cursor_sqlite.fetchone()
        if (
            len(result) == 1
            and result[0] is not None
        ):
            latest_update = str(result[0])
        
    except Exception as e:
        print(e)

    print(latest_update, '\n')
    
    with open('./sierra_patron_data.sql') as f:
        cursor.execute(f.read(), (latest_update,))

    with open('./output.csv', 'w') as f:
        writer = csv.writer(f)
        
        i = 0
        while(data:=cursor.fetchmany(1000)):
            if i==0:
                columns = [col[0] for col in cursor.description]
                writer.writerow(columns)
    
            # write the data to a csv
            writer.writerows(data)

            # insert the appropriate data into each local sqlite table
            
            # patrons table
            con_sqlite.executemany(
                sql_patron_insert, 
                (
                    (
                        row[columns.index('patron_record_id')],
                        row[columns.index('patron_record_num')],
                        row[columns.index('campus_code')],
                        row[columns.index('barcode1')],
                        row[columns.index('home_library_code')],
                        row[columns.index('ptype_code')],
                        row[columns.index('create_date')],
                        row[columns.index('delete_date')],
                        row[columns.index('update_date')],
                        row[columns.index('expire_date')],
                        row[columns.index('active_date')],
                        row[columns.index('claims_returned_total')],
                        row[columns.index('owed_amt_cents')],
                        row[columns.index('mblock_code')],
                        row[columns.index('highest_level_overdue_num')],
                        row[columns.index('num_revisions')],
                    )
                    for row in data
                )
            )

            # patron_address_json table
            con_sqlite.executemany(
                sql_address_json_insert, 
                (
                    (
                        row[columns.index('patron_record_id')],
                        str(row[columns.index('patron_address_json')]),
                    )
                    for row in data
                )
            )

            
    
            i+=1
            print('.', end='')
            # break
        
    print('done. ', i)
    con_sqlite.commit()
    cursor_sqlite.close()
    cursor.close()

2023-08-10 08:22:46 

...................................done.  35
